# Convolutional Neural Network

### Importing the libraries

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [72]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [73]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [74]:
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8002 images belonging to 2 classes.


In [75]:
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2021 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [76]:
cnn = Sequential()

### Step 1 - Convolution

In [77]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))

### Step 2 - Pooling

In [78]:
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [79]:
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [80]:
cnn.add(Flatten())

### Step 4 - Full Connection

In [81]:
cnn.add(Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [82]:
cnn.add(Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [83]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [92]:
cnn.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=50,
    validation_data=test_set,
    validation_steps=len(test_set)
)

Epoch 1/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 118s 462ms/step - accuracy: 0.9731 - loss: 0.0715 - val_accuracy: 0.8060 - val_loss: 0.8117
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 95us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 119s 465ms/step - accuracy: 0.9729 - loss: 0.0722 - val_accuracy: 0.7872 - val_loss: 0.9747
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 133s 455ms/step - accuracy: 0.9747 - loss: 0.0704 - val_accuracy: 0.8001 - val_loss: 0.9852
Epoch 6/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 90us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 116s 456ms/step - accuracy: 0.9724 - loss: 0.0737 - val_accuracy: 0.8031 - val_loss: 0.8821
Epoch 8/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 120us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 112s 436ms/step - accuracy: 0.9718 - loss: 0.0748 -

In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_set)
print(f"Test Accuracy: {test_accuracy:.2f}")

## Part 4 - Making a single prediction

In [ ]:
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale

    prediction = cnn.predict(img_array)
    if prediction[0][0] > 0.5:
        print("Prediction: Dog")
    else:
        print("Prediction: Cat")

In [ ]:
predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg')
predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_2.jpg')

In [ ]:
cnn.save('cat_dog_classifier.h5')